In [1]:
from darts.models import (LinearRegressionModel,
                          CatBoostModel, 
                          LightGBMModel, 
                          Prophet, 
                          RandomForest, 
                          XGBModel,
                          ARIMA,
                          AutoARIMA,
                          Theta)

# model_loaded = LinearRegressionModel.load("../model/linearRegressionModel_totalOrder_14Jun67.pkl")

c:\Users\User\anaconda3\envs\conda_dart\Lib\site-packages\statsforecast\utils.py:231: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from darts import TimeSeries

#file name data "date"-1 
df = pd.read_csv('../csv/Ming25Jun67_total_order.csv')
df.tail()

,order_completed_at,unique_order_count
536,2024-06-20,120
537,2024-06-21,152
538,2024-06-22,138
539,2024-06-23,126
540,2024-06-24,149


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_completed_at  541 non-null    object
 1   unique_order_count  541 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.6+ KB


In [15]:
df.shape

(541, 2)

In [16]:
# predict future
# make timeseries Object
from darts import TimeSeries
series = TimeSeries.from_dataframe(df,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'])

In [17]:
# model_loaded.fit(series)

In [18]:
# forecast = model_loaded.predict(7)
# forecast = forecast.pd_dataframe().reset_index()
# forecast.columns = ['Date', 'Total Order']
# forecast.tail(7)

In [19]:
# model_loaded.save("../model/linearRegressionModel_totalOrder_16Jun67.pkl")

# Loop Predict and Save to csv

In [5]:
from darts import TimeSeries
from darts.models import LinearRegressionModel, ExponentialSmoothing, CatBoostModel, LightGBMModel, Prophet, RandomForest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#file name data "date"-1 
df = pd.read_csv('../csv/Ming25Jun67_total_order.csv')
print(df.tail())

from darts import TimeSeries
series = TimeSeries.from_dataframe(df,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")




    order_completed_at  unique_order_count
536         2024-06-20                 120
537         2024-06-21                 152
538         2024-06-22                 138
539         2024-06-23                 126
540         2024-06-24                 149


In [6]:
# Function to replace outliers with the mean of the rest of the values
from scipy.stats import zscore
from darts import TimeSeries

def replace_outliers_with_mean(df, column_name):
    # Calculate Z-scores
    z_scores = zscore(df[column_name])
    
    # Identify outliers (using a threshold of 3 for Z-score)
    outliers = np.abs(z_scores) > 3
    
    # Calculate mean of non-outliers
    mean_non_outliers = df.loc[~outliers, column_name].mean()
    
    # Replace outliers with the mean of non-outliers
    df.loc[outliers, column_name] = mean_non_outliers
    
    return df

# Replace outliers in 'gmv' column
df_remove_out = replace_outliers_with_mean(df, 'unique_order_count')
series_remove_outlier = TimeSeries.from_dataframe(df_remove_out,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")

C:\Users\User\AppData\Local\Temp\ipykernel_35672\2557321846.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '142.47269303201506' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[outliers, column_name] = mean_non_outliers


In [7]:
series = series_remove_outlier

In [8]:
from darts.dataprocessing.transformers import (Scaler,)

scaler = Scaler()
series_rescaled = scaler.fit_transform(series)

In [9]:
series_rescaled.head()

<TimeSeries (DataArray) (order_completed_at: 5, component: 1, sample: 1)> Size: 40B
array([[[0.00990099]],

       [[0.04356436]],

       [[0.15643564]],

       [[0.12475248]],

       [[0.11287129]]])
Coordinates:
  * order_completed_at  (order_completed_at) datetime64[ns] 40B 2023-01-01 .....
  * component           (component) object 8B 'unique_order_count'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [11]:

# Import module
import pickle

# loaded_model = pickle.load(open("", 'rb'))

def load_model_predict(path: str, timeseries, name: str):
    loaded_model = pickle.load(open(path, 'rb'))
    loaded_model.fit(timeseries)
    loaded_model.save(path)

    forecast = loaded_model.predict(7)
    forecast  = scaler.inverse_transform(forecast)
    forecast = forecast.pd_dataframe().reset_index()
    forecast.columns = ['Date', 'Total Order']

    print(name)
    print(forecast)
    print()

    #!!! check date
    date = "2024-06-25"
    forecast.tail(7).to_csv(f'../forecast/{date}_{name}.csv', index=False)
    return forecast.tail(7)


import os
# Assign directory
directory = r"D:\Ming\ProgramLang_And_IDE\Python67\pythonEnv_darts\model"
out_df = pd.DataFrame()
first_col_date = True
# Iterate over files in directory
for name in os.listdir(directory):

    # Get the full file path
    file_path = os.path.join(directory, name)

    # Print the file path
    print(file_path)
    predict_df= load_model_predict(file_path, series_rescaled, name)
    if first_col_date: 
        out_df["Date"] = predict_df["Date"];first_col_date=False
    out_df[name] = predict_df["Total Order"]

date = "2024-06-25"
out_df.tail(7).to_csv(f'../forecast/{date}.csv', index=False)
 

D:\Ming\ProgramLang_And_IDE\Python67\pythonEnv_darts\model\ARIMA_Model_totalOrder_25Jun67.pkl
ARIMA_Model_totalOrder_25Jun67.pkl
        Date  Total Order
0 2024-06-25   138.198611
1 2024-06-26   138.214333
2 2024-06-27   138.229431
3 2024-06-28   138.243930
4 2024-06-29   138.257853
5 2024-06-30   138.271223
6 2024-07-01   138.284062

D:\Ming\ProgramLang_And_IDE\Python67\pythonEnv_darts\model\AutoARIMA_Model_totalOrder_25Jun67.pkl
AutoARIMA_Model_totalOrder_25Jun67.pkl
        Date  Total Order
0 2024-06-25   136.855570
1 2024-06-26   136.467307
2 2024-06-27   134.847501
3 2024-06-28   134.364410
4 2024-06-29   134.032632
5 2024-06-30   133.883318
6 2024-07-01   133.801847

D:\Ming\ProgramLang_And_IDE\Python67\pythonEnv_darts\model\CatBoostModel_totalOrder_25Jun67_31lags.pkl
CatBoostModel_totalOrder_25Jun67_31lags.pkl
        Date  Total Order
0 2024-06-25   136.368804
1 2024-06-26   133.652766
2 2024-06-27   127.727396
3 2024-06-28   136.278871
4 2024-06-29   136.059137
5 2024-06-30 